In [5]:
import datetime
import json
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit_experiments.framework import ExperimentEncoder
from qiskit_research.mzm_generation import (
    KitaevHamiltonianExperiment,
    KitaevHamiltonianExperimentParameters,
)
from qiskit_research.mzm_generation.utils import transpile_circuit
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations

from qiskit.circuit.library import RZGate
from qiskit_research.utils.dynamical_decoupling import get_instruction_durations

from qiskit_experiments.framework import BatchExperiment
from qiskit.visualization.timeline import draw

In [2]:
# get backend

# uncomment to use hardware backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend_name = 'ibmq_guadalupe'

# Aer simulator backend
# provider = None
# backend_name = "aer_simulator"

In [3]:
# experiment parameters
n_modes = [9]
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = list(np.linspace(0.0, 3.0, num=5))
# dynamical_decoupling_sequences = None
dynamical_decoupling_sequences=[None, 'X2pm', 'XY4pm', 'XY8pm']
num_dd_passes=1
uhrig_spacing = True
concat_layers = 2
shots = 10000
readout_calibration_shots = 10000

experiments = []

for n in n_modes:
    # pick qubits
    qubits, backend_name, error_score = pick_qubit_layout(n, backend_name, provider)
    print(f"n = {n} Using qubits {qubits}.")

    occupied_orbitals_list = list(orbital_combinations(n, threshold=2))

    # date
    date = datetime.datetime.now().isoformat()

    # create experiment
    params = KitaevHamiltonianExperimentParameters(
        timestamp=date,
        backend_name=backend_name,
        qubits=qubits,
        n_modes=n,
        tunneling_values=tunneling_values,
        superconducting_values=superconducting_values,
        chemical_potential_values=chemical_potential_values,
        occupied_orbitals_list=occupied_orbitals_list,
        dynamical_decoupling_sequences=dynamical_decoupling_sequences,
        num_dd_passes=num_dd_passes,
        uhrig_spacing=uhrig_spacing,
        concat_layers=concat_layers,
        seed=11364,
    )
    experiment = KitaevHamiltonianExperiment(params, provider)
    experiments.append(experiment)

    print(f"Total number of circuits for n = {n}: {len(experiment.circuits())}")

n = 9 Using qubits [2, 3, 5, 8, 11, 14, 13, 12, 15].
Total number of circuits for n = 9: 2520


In [9]:
draw(experiment._transpiled_circuits()[0])

CircuitError: 'qubit_stop_time undefined. Circuit must be scheduled first.'

In [10]:
%%time
# schedule readout calibration
for experiment in experiments:
    n = experiment.params.n_modes
    print("Scheduling readout calibration n = {n}...")
    mit = mthree.M3Mitigation(experiment.backend)
    mit.cals_from_system(experiment.params.qubits, shots=readout_calibration_shots, async_cal=True)

    # schedule experiment jobs
    print(f"Scheduling experiment n = {n}...")
    # limit number of circuits per job to avoid timeouts
    experiment.backend.configuration().max_experiments = 300
    data = experiment.run(shots=shots)
    # if provider:
        # this only works if the Qiskit Experiments database service is available
        # data.save()

    # save readout calibration
    filename = f"data/readout_calibration/{experiment.backend.name()}/{date}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print(f"Waiting for readout calibration data ...")
    mit.cals_to_file(filename)
    print(f"Readout calibration saved.")

    data.block_for_results()

    # save data
    filename = f"data/{params.filename}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        json.dump(data, f, cls=ExperimentEncoder)
    print(params.filename)

Scheduling readout calibration n = {n}...
Scheduling experiment n = 9...
Waiting for readout calibration data ...
Readout calibration saved.
2022-06-06T18:57:38.701872_ibmq_guadalupe_n9
CPU times: user 4min 44s, sys: 55.1 s, total: 5min 39s
Wall time: 2h 44min 12s


In [8]:
data.block_for_results()

# save data
filename = f"data/{params.filename}.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "w") as f:
    json.dump(data, f, cls=ExperimentEncoder)

print(params.filename)

jobs and analysis raised exceptions [Experiment ID: 6f8d4ab8-c64e-4ef1-b603-aaed1ddc7158]:Traceback (most recent call last):

  File "/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(

  File "/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):

  File "/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):

socket.gaierror: [Errno 8] nodename nor servname provided, or not known


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/georgezhou/miniforge3/envs/DDResearch/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlop

2022-06-06T14:36:55.135537_ibmq_guadalupe_n9
